# 1. Using huggingface Transformer

In [ ]:
!pip install transformers==4.34.0

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch
import gc

model_id = "../base_models/amazon/MistralLite"
#model_id = "../base_models/mistralai/Mistral-7B-Instruct-v0.1"

/home/ec2-user/SageMaker/envs/mistral/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## use flash attention2

In [ ]:
!pip install flash-attn==2.3.1.post1 --no-build-isolation

In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.bfloat16,
                                             use_flash_attention_2=True,
                                             device_map="auto",)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.89s/it]


In [48]:
sequences = pipeline(
   "<|prompter|>In the yard were chickens and rabbits. A chicken has 1 head and 2 legs while a rabbit has 1 head and 4 legs. Together they had 23 heads and 76 legs. How many chickens and how many rabbits were in the yard? Think step by step. </s><|assistant|>",
   #"<s>[INST] At a zoo, Pascal saw some peacocks and deer. He counted 18 heads and 52 legs altogether. How many animals of each type did Pascal see? [/INST]",
    #"<s>[INST] What is the longest river in the world? [/INST]",
    max_new_tokens=600,
    do_sample=False,
    return_full_text=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"{seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


# 2. Use TGI docker container

# 3. Use vLLM docker container

# 4. Deploy on AWS Sagemaker

In [1]:
!pip install -U sagemaker==2.192.1

  Using cached sagemaker-2.192.1-py2.py3-none-any.whl
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.183.0
    Uninstalling sagemaker-2.183.0:
      Successfully uninstalled sagemaker-2.183.0


In [2]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region,
 version="1.1.0"
)

In [4]:
image_uri

'763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04'

In [ ]:
model_name = "gpt-neox-20b-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

hub = {
    'HF_MODEL_ID':'EleutherAI/gpt-neox-20b',
    'HF_TASK':'text-generation',
    'SM_NUM_GPUS':'4',
    'HF_MODEL_QUANTIZE':'true'
}

model = HuggingFaceModel(
    name=model_name,
    env=hub,
    role=role,
    image_uri=image_uri
)